In [5]:
import json
import os

with open('../data/category.json', 'r') as f:
    classes = json.load(f)

labels_dir = 'labels/test'

for file in os.listdir(labels_dir):
    if file.endswith('.txt'):
        file_path = os.path.join(labels_dir, file)

        with open(file_path, 'r') as f:
            lines = f.readlines()

            filtered_lines = [
                line for line in lines
                if line.strip() and int(line.split()[0]) in classes
            ]

        with open(file_path, 'w') as f:
            f.writelines(filtered_lines)


In [4]:
import yaml

data  = """
names:
  0: shirt, blouse
  1: top, t-shirt, sweatshirt
  2: sweater
  3: cardigan
  4: jacket
  5: vest
  6: pants
  7: shorts
  8: skirt
  9: coat
  10: dress
  11: jumpsuit
  12: cape
  13: glasses
  14: hat
  15: headband, head covering, hair accessory
  16: tie
  17: glove
  18: watch
  19: belt
  21: tights, stockings
  22: sock
  23: shoe
  24: bag, wallet
  25: scarf
  27: hood
  38: bow
  39: flower
  43: ruffle

path: /kaggle/input/fashionpedia-dataset
test: images/test
train: images/train
val: images/test
"""

with open('data2.yaml', 'w') as f:
    yaml.dump(yaml.safe_load(data), f)

In [9]:
import os
import json

with open('../data/category.json', 'r') as f:
    classes = json.load(f)

remap_dict = {old: new for new, old in enumerate(classes)}

labels_dir = 'labels/val'

for filename in os.listdir(labels_dir):
    if filename.endswith(".txt"):
        file_path = os.path.join(labels_dir, filename)

        with open(file_path, "r") as file:
            lines = file.readlines()

        remapped_lines = []
        for line in lines:
            parts = line.strip().split()
            if not parts:
                continue
            old_class_id = int(parts[0])
            new_class_id = remap_dict.get(old_class_id)
            if new_class_id is not None:
                remapped_line = " ".join([str(new_class_id)] + parts[1:]) + "\n"
                remapped_lines.append(remapped_line)

        # Overwrite file with remapped labels
        with open(file_path, "w") as file:
            file.writelines(remapped_lines)

In [9]:
import os
import pandas as pd
import requests
from tqdm import tqdm

# Load the catalog if not already loaded
catalog = pd.read_csv('/home/pchhalotre/Desktop/flickd-hackathon/backend/data/catalog.csv')

for _, row in tqdm(catalog.iterrows(), total=len(catalog)):
    product_id = str(row['id'])
    image_url = row['image_url']
    dest_dir = os.path.join('images', product_id)
    os.makedirs(dest_dir, exist_ok=True)
    image_name = os.path.basename(image_url.split('?')[0])
    dest_path = os.path.join(dest_dir, image_name)

    if not os.path.exists(dest_path):
        try:
            response = requests.get(image_url, timeout=10)
            if response.status_code == 200:
                with open(dest_path, 'wb') as f:
                    f.write(response.content)
        except Exception as e:
            print(f"Failed to download {image_url}: {e}")

  2%|▏         | 193/8312 [01:44<7:53:00,  3.50s/it]

Failed to download https://cdn.shopify.com/s/files/1/0785/1674/8585/files/WEB2_86f285a3-976b-46ee-8035-af30855b80d3_1600x.jpg?v=1739271707: HTTPSConnectionPool(host='cdn.shopify.com', port=443): Max retries exceeded with url: /s/files/1/0785/1674/8585/files/WEB2_86f285a3-976b-46ee-8035-af30855b80d3_1600x.jpg?v=1739271707 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7eed3f5820c0>: Failed to establish a new connection: [Errno 101] Network is unreachable'))


  2%|▏         | 197/8312 [02:00<10:08:05,  4.50s/it]

Failed to download https://cdn.shopify.com/s/files/1/0785/1674/8585/files/DSCF1656_f0ec2a6e-f9f5-4257-8c16-bb86462434b9_1600x.jpg?v=1738831884: HTTPSConnectionPool(host='cdn.shopify.com', port=443): Max retries exceeded with url: /s/files/1/0785/1674/8585/files/DSCF1656_f0ec2a6e-f9f5-4257-8c16-bb86462434b9_1600x.jpg?v=1738831884 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7eed3f580710>: Failed to establish a new connection: [Errno 101] Network is unreachable'))


  5%|▍         | 398/8312 [04:02<8:51:55,  4.03s/it] 

...

Failed to download https://cdn.shopify.com/s/files/1/0785/1674/8585/files/13thNOV2024Virgio-0312_1600x.jpg?v=1734527231: HTTPSConnectionPool(host='cdn.shopify.com', port=443): Max retries exceeded with url: /s/files/1/0785/1674/8585/files/13thNOV2024Virgio-0312_1600x.jpg?v=1734527231 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7eed3f57d040>: Failed to establish a new connection: [Errno 101] Network is unreachable'))


 14%|█▍        | 1191/8312 [12:17<9:53:38,  5.00s/it]

Failed to download https://cdn.shopify.com/s/files/1/0785/1674/8585/files/4thDEC2024Virgio-0251_1600x.jpg?v=1738668004: HTTPSConnectionPool(host='cdn.shopify.com', port=443): Max retries exceeded with url: /s/files/1/0785/1674/8585/files/4thDEC2024Virgio-0251_1600x.jpg?v=1738668004 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7eed3f587800>: Failed to establish a new connection: [Errno 101] Network is unreachable'))


 16%|█▌        | 1313/8312 [13:21<1:11:10,  1.64it/s]


KeyboardInterrupt: 